# **GROUP 3 PROJECT REPORT**

# Introduction


Wine is a popular alcoholic beverage that is enjoyed around the world and can have its origins traced back thousands of years. Throughout history, people have developed many methods and technologies for creating this luxury product since a wine's quality is influenced by a variety of factors such as the type of grape, region where it is produced, weather, and wine-making techniques. Winemakers strive to produce high-quality wine that appeals to consumers' taste buds; however, the assessment of wine quality can be subjective and varies from person to person depending on their taste. Of the 5 senses we have available to us, taste is often thought of as the least understood sense because of how difficult it is to quantify. This can pose problems for industries where business is influenced by the taste of a product, which is especially true for the wine industry. Expert wine testers are often needed to determine and certify the quality of different wines through sensory tests. (Cortez, 2009) While this is the current industry standard, are there any other ways to certify the quality of a wine without depending on these experts? This leads to our question:

**Is it possible to predict the quality classification of a wine based on quantifiable physicochemical properties using data analysis?**


We used the Wine Quality dataset, which is publicly available on the UCI Machine Learning Repository. This dataset contains details of two separate datasets, red and white *winho verde* wine samples from the north of Portugal. The red wine dataset contains 1,599 samples and the white dataset contains 4,898 samples. There are 11 quantified physicochemical properties (e.g., fixed acidity, volatile acidity, citric acid, residual sugar, residual sugar, chlorides, free sulfur dioxide, total sulfur dioxide, density, pH, sulphates, and alcohol) and there is one categorical variable for the wine quality classified on an integer scale of 1-10. In this project, we concatenated the two datasets to create a single dataset containing 6,497 samples and including a new categorical variable for the wine type. We randomly split this dataset into a training set (75% of the data) and a test set (25% of the data).

<u> Dataset citation: </u> \
Cortez, P., Cerdeira, A., Almeida, F., Matos, T., & Reis, J. (2009). Modeling wine preferences by data mining from physicochemical properties. *Decision Support Systems*, 47(4), 547–553. doi:10.1016/j.dss.2009.05.016

# Methods & Results:

**TODO LIST**

- describe in written English the methods you used to perform your analysis from beginning to end that narrates the code the does the analysis.

your report should include code which:
- loads data from the original source on the web 
- wrangles and cleans the data from it's original (downloaded) format to the format necessary for the planned analysis
- performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis 
- creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis
- performs the data analysis
- creates a visualization of the analysis 
- note: all tables and figure should have a figure/table number and a legend

Please refer to the Code.ipynb file.  
Note. the numbers do not correlate.

==========================================================================================================================

**1. LOADING DATA**

* First we set up all of the packages we will be using for the analysis. Note that a more recent version of altair must be loaded in to prevent a bug specific to jupyter notebooks and the use of hconcat or vconcat.

In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.compose import make_column_transformer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV,
    RandomizedSearchCV
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier

! pip install altair==5.0.0rc1 
import altair as alt

<br>

* The wine data for red and white wine is then loaded from the web using the pandas library's read_csv() function. The URL of each dataset was provided and contains data on wine quality, wine type, and various chemical properties of the wine. Specifically, the Red Wine data was set to data_red and White Wine data was set to data_white. Furthermore, we concatenated red and white wine data and set it to data_wine.


In [2]:
data_red = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=";").assign(type = "red")
data_white = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv", sep=";").assign(type = "white")
data_wine = pd.concat([data_red, data_white])
data_wine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,white
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,white
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,white
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,white


*Table: 1*

<br>

**2. SUMMARY OF DATA**

* We then preform a summary of the data so that we may know what we are working with in our exploratory analysis. First, we explore the statistical data of each column in the dataset which includes the count, mean, standard deviation, minimum, and maximum values of each chemical property. We then look for how many null values are in each column, and finally we count the number of unique occurances in each column. This summary of data was preformed on the concatenated dataset of red and white wine, so it explores all data availible.

In [3]:
data_wine.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000
mean,7.215307,0.339666,0.318633,5.443235,0.056034,30.525319,115.744574,0.994697,3.218501,0.531268,10.491801,5.818378
std,1.296434,0.164636,0.145318,4.757804,0.035034,17.749400,56.521855,0.002999,0.160787,0.148806,1.192712,0.873255
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000


*Table: 2*

In [4]:
data_wine.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
type                    0
dtype: int64

*Array: 1*

In [5]:
data_wine.nunique()

fixed acidity           106
volatile acidity        187
citric acid              89
residual sugar          316
chlorides               214
free sulfur dioxide     135
total sulfur dioxide    276
density                 998
pH                      108
sulphates               111
alcohol                 111
quality                   7
type                      2
dtype: int64

*Array: 2*

<br>

* *Array 1* shows us that there are no observations containing null values, which will help us later in our analysis. Also, *Array 2* shows that there are 7 unique occurrences in the quality column and *Table 2* shows us that the min and max values of the quality column are 3 and 9 respectively. This indicates that all wines in the dataset will have a quality in the range of 3 to 9, meaning that a knn model will have a hard time predicting values outside of that range. We can also see from *Table 2* that the mean of the quality column is 5.8 with a standard deviation of 0.87.

* We further explore the quality column with a bar plot of occurrences:

In [6]:
sample_size = 5000
data_sample = data_wine.sample(sample_size, random_state=123)

quality_plot = (alt.Chart(data_sample, title=alt.TitleParams("Distribution of Quality", fontSize=20))
                .mark_bar(size=25)
                .encode(
                    x=alt.X("quality:Q", title="Quality", scale=alt.Scale(domain=[2,9], zero=False)),
                    y=alt.Y("count()", title="Count of Occurances", scale=alt.Scale(zero=False)))
               .properties(
               width=400,
               height=400))
quality_plot

alt.Chart(...)

*Graph: 1*

* *Graph 1* gives us a good idea of how the quality ratings in our dataset are distributed


* We will now perform a visual summary to compare the relevancy of each chemical property to the quality of the wine. We will also include a visualization of wine type vs quality to see if there is a significant difference between the two types. Note that will be taking a sample to make our graphs more readable.

In [7]:
scatter_plots = []
sample_opacity = 0.7
sample_size = 450

data_sample = data_wine.sample(sample_size, random_state=123)
red_count   = data_sample[data_sample["type"] == "red"].shape[0]      #for subtitle
white_count = data_sample[data_sample["type"] == "white"].shape[0]    #for subtitle

for property_name in data_sample.columns[:-2]:
    scatter_plot = (alt.Chart(data_sample)
                    .mark_circle(opacity=sample_opacity)
                    .encode(
                        x=alt.X(f"quality:Q", scale=alt.Scale(domain=(1, 10))),
                        y=alt.Y(f"{property_name}:Q", scale=alt.Scale(zero=False)),
                        color="type:N")
                    .properties(
                        width=250,
                        height=250,
                        title=f"{property_name.title()} vs Quality, by Wine Type"))
    scatter_plots.append(scatter_plot)
    
scatter_plots.append(alt.Chart(data_sample)
                     .mark_circle(opacity=sample_opacity)
                     .encode(
                         x=alt.X(f"quality:Q", scale=alt.Scale(domain=[1,10])),
                         y=alt.Y("type"),
                         color="type:N")
                     .properties(
                         width=250,
                         height=250,
                         title=f"Type vs Quality, by Wine Type"))
 
    
wine_plot = alt.VConcatChart(vconcat=(alt.hconcat(*scatter_plots[0:4]), 
                                      alt.hconcat(*scatter_plots[4:8]), 
                                      alt.hconcat(*scatter_plots[8:])), 
                             title=alt.TitleParams(
                                 "Plot of All Wine Properties vs Quality", 
                                 fontSize=30,
                                 subtitle="(sample size "+str(sample_size)+": "+str(red_count)+" red / "+str(white_count)+" white)", anchor="middle", dy=-10))

wine_plot

alt.VConcatChart(...)

*Graph: 2*

* *Graph 2* shows us that there are not many chemical properties with strong positive or negative relationships with quality. The strongest relationship of them all seems to be a slight positive relationship between alcohol and quality. We can also see that quality does indeed range between 3 and 9 for both red and white wines, though it seems that quality ratings of 3 and 9 are rare since none/a small amount of them occurred in this sample.

* In light of this, we will use all chemical properties in our analysis in the hopes that there is a correlation that cannot be seen with this 2D representation. We will also test our models in 2 different ways: once with the concatenated dataset, and once with the red wine and white wine datasets separated. In this way, we can compare the accuracies and we will know what method of training is best.

* Finally, we will preform a visual analysis of the ratio between red and white wine occurances in our dataset. Again we limit ourselves to a sample size of 5000 so that altair can work with our data

In [8]:
sample_size = 5000
data_sample = data_wine.sample(sample_size, random_state=2020)

red_count   = data_sample[data_sample["type"] == "red"].shape[0]      #for subtitle
white_count = data_sample[data_sample["type"] == "white"].shape[0]    #for subtitle

type_plot = (alt.Chart(data_sample, title=alt.TitleParams("Wine Type Distribution", 
                                                         fontSize=20,
                                                         subtitle=str(red_count)+" red / "+str(white_count)+" white"))
                .mark_bar(size=70)
                .encode(
                    x=alt.X("type", title="Wine Type"),
                    y=alt.Y("count()", title="Count of Occurances"),
                    color="type:N")
               .properties(
               width=250,
               height=250))

type_plot

alt.Chart(...)

*Graph: 3*

**3. PREPROCESSING DATA**

* First we split the data into its X/y portions and its test/train portions. We do this separately for the concatenated data, the red wine data, and the white wine data. We also haven't fully preprocessed this data yet, so the X portions are labelled as "raw"

In [9]:
# splits for concatenated wine data
X = data_wine.drop('quality', axis=1)
y = data_wine['quality']
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

# splits for red wine data
X_red = data_red.drop(['quality', 'type'], axis=1)
y_red = data_red['quality']
X_train_red_raw, X_test_red_raw, y_train_red, y_test_red = train_test_split(X_red, y_red, test_size=0.25, random_state=123)

#splits for white wine data
X_white = data_white.drop(['quality', 'type'], axis=1)
y_white = data_white['quality']
X_train_white_raw, X_test_white_raw, y_train_white, y_test_white = train_test_split(X_white, y_white, test_size=0.25, random_state=123)

* After this we will preprocess and scale the raw X data for each dataset. For the concatenated dataset we will also use OneHotEncoder so that we can incorporate the type of wine as a numerical feature in our models. We also know from *Array 1* that there are no missing values in our data, so we don't have to remove any in our preprocessing step.

In [10]:
features_categorical = [
    "type"
]
features_numerical = [
    'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
    'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
    'pH', 'sulphates', 'alcohol'
]


# concatenated wine data
ct = make_column_transformer(
    (StandardScaler(), features_numerical),
    (OneHotEncoder(drop="first"), features_categorical)
)
X_train = ct.fit_transform(X_train_raw)

# red and white wine data
ct = make_column_transformer(
    (StandardScaler(), features_numerical),
)
X_train_red = ct.fit_transform(X_train_red_raw)

X_train_white = ct.fit_transform(X_train_white_raw)

* The data is now completely prepared for our exploratory analysis.

<br>

**4. DATA ANALYSIS**

* The main question we sought to answer was: 
> "Is it possible to predict the quality classification of a wine based on quantifiable physicochemical properties using data analysis?" 

* To answer this question, we decided to use five different machine learning models to predict wine quality based on the chemical properties of the wine. **We decided to do this after reading about a similar analysis done by NEED THE CITATION

**this is where I left off, I'll keep working on it sat**


4. Data Analysis:
    The main question we sought to answer was: "What are the most important factors affecting wine quality?" To answer this question, we used a machine learning model to predict wine quality based on the chemical properties of the wine.
    
The following steps were taken:

- The dataset was split into training and testing sets using the train_test_split() function from sklearn.model_selection.
- A OneHotEncoder was used to transform the 'type' column to a numerical format.
- A ColumnTransformer was used to apply the encoder to the training data.
- A RandomForestRegressor model was trained on the training data.
- The trained model was used to predict wine quality for the testing data.
- The predictions were evaluated using the mean squared error (MSE) and R-squared values.


(ToDo): summary of dataset, visualization of dataset, visulization of analysis 

# Discussion:
summarize what you found
- discuss whether this is what you expected to find?
- discuss what impact could such findings have?
- discuss what future questions could this lead to?

In this project, I explored the relationship between wine properties and wine quality using the Wine Quality dataset. The dataset contained 11 columns describing different properties of wine, such as fixed acidity, volatile acidity, citric acid, residual sugar, chlorides, free sulfur dioxide, total sulfur dioxide, density, pH, sulphates, and alcohol. The target variable was wine quality, which was a score between 1 and 10. The dataset contained 6497 observations, and after cleaning the data, I had 5367 observations left.

............. write more about what the wine score was after data analysis (distribution), how the visualizations looked like for what relationships,  what machine models we used and what the predict score was. ..........

...........then write about what we found. which idk column features were dominating the wine quality results.  ..........

The findings of this project could have several impacts. First, they could be used by wine producers to optimize their production processes and create higher-quality wines. By focusing on the wine properties that are most strongly related to quality, producers could make more informed decisions about which grapes to use and how to process them. Additionally, the findings could be used to educate consumers about the factors that contribute to wine quality and help them make more informed purchasing decisions.

The findings of this project could also lead to several future research questions. For example, future studies could explore the impact of other wine properties, such as tannins and acidity, on wine quality. Additionally, future studies could explore the relationship between wine properties and the sensory experience of drinking wine, such as flavor and aroma. Finally, future studies could investigate the impact of different winemaking techniques on wine quality and the relationship between wine properties and aging.

# References:
- At least 2 citations of literature relevant to the project (format is your choice, just be consistent across the references).
- Make sure to cite the source of your data as well.

# Methods


We will be performing a classification using n nearest neighbours to predict the quality of observed *winho verde* wines. We will conduct our data analysis using all properties provided by the data frame. We will do this because by looking at the graphs made during our Preliminary Exploratory Data Analysis, we found that there weren't many strong relationships between wine properties and quality--or at least these relationships haven't made themselves immediately apparent. This is only to the human eye, however, so we hope that once all properties are analyzed together (something which can't be represented by a 2-D graph) that patterns will make themselves more apparent to our classification model. We also wish to include the wine type in the properties we use in our model. How we go about this though is something to be seen during the creation of our project, since there are two ways we can think of doing this: 

1) We turn the categorical data of wine type into numerical data using OneHotEncoder() and incorporate that into our n_nearest_neighbors model. We aren't sure if this will work or if it will confuse our model since both wine types can have very different physiochemical makeups. This is best demonstrated by our residual sugar graph. If this does seem to confuse our model and our model's accuracy is low, then we may test our second method.

2) We keep our data separated into two datasets, one for red and one for white, then train both separately and create a classification model for each type. This might help our model avoid any confusion between the differing physiochemical makeups of the two wine types.

We will visualize our result by creating a scatterplot of {property} vs quality for each property using our testing data (or a sample of our testing data) and then overlaying a line plot of our predicted quality. It will look similar to the graph layout in our Preliminary Exploratory Data Analysis, just including the overlay. We would also like to include a visualization involving our prediction accuracy, but we are unsure how to go about it.

# Expected Outcomes and Significance


We expect that our model(s) will accurately predict qualities between 4 and 7 since that's where most of the training data falls. It may have trouble predicting wines with quality far outside that range. 

The accurate predictions of wine quality based on physicochemical properties could have large implications for how the wine industry conducts its quality certifications. It is doubtful that our model will have a high enough accuracy to instantly revolutionize the wine certification process, but a small success with our method could encourage further research with more complicated methods. If those more complicated methods can prove themselves trustworthy in predicting wine quality, then that could change the industry standard for how wines are certified in the future (assuming people allow this change).

The success of this scenario could lead to future questions about the relationship between the physicochemical properties of various foods and what people think tastes "good".

# Discussion:
- summarize what you found
- discuss whether this is what you expected to find?
- discuss what impact could such findings have?
- discuss what future questions could this lead to?


# References:
- At least 2 citations of literature relevant to the project (format is your choice, just be consistent across the references).
- Make sure to cite the source of your data as well.